## Load latest headlines

In [1]:
# ========== Bootstrap: ensure required Python packages are present =========
import importlib, subprocess, sys
from typing import Optional

def _ensure(pkg_name: str, import_name: Optional[str] = None):
    try:
        importlib.import_module(import_name or pkg_name)
    except ModuleNotFoundError:
        subprocess.check_call([sys.executable, '-m', 'pip', 'install', pkg_name])
    finally:
        globals()[import_name or pkg_name] = importlib.import_module(import_name or pkg_name)

_ensure('pandas')
print('All dependencies ready.\n')


All dependencies ready.



In [2]:
from pathlib import Path
archive = Path('archive')
archive.mkdir(exist_ok=True)
import pandas as pd
latest = pd.read_csv('../headlines/latest.csv')
latest.head()

,pubdate,source,title,link
0,2025-10-15-03-51-04 +0000,bbc,Bitcoin worth $14bn seized in US-UK crackdown ...,https://www.bbc.com/news/articles/c70jw436n0yo...
1,2025-10-15-03-49-12 +0000,nypost,Shopper stabbed by irate woman in NJ parking l...,https://nypost.com/2025/10/14/us-news/woman-st...
2,2025-10-15-03-47-00 +0000,wsj,China’s Deflationary Pressures Ease Slightly,https://www.wsj.com/economy/chinas-deflationar...
3,2025-10-15-03-42-21 +0000,bbc,British social media star 'Big John' detained ...,https://www.bbc.com/news/articles/cwy196k9p4po...
4,2025-10-15-03-26-25 +0000,nyt,Suspect Pleads Guilty in Firebombing of Pennsy...,https://www.nytimes.com/2025/10/14/us/arson-ha...


## Calculate word frequencies

In [3]:
import re
from collections import Counter
from datetime import datetime

with open('exclude.txt') as f:
    stop_words = set(w.strip() for w in f if w.strip())
words = re.findall(r'[A-Za-z]+', ' '.join(latest['title']).lower())
filtered = [w for w in words if w not in stop_words and len(w) > 1]
counts = Counter(filtered)
score_df = (
    pd.DataFrame(counts.items(), columns=['word','score'])
    .sort_values('score', ascending=False)
)
score_df[['score','word']].to_csv('scores.csv', index=False)
timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')
score_df[['score','word']].to_csv(archive / f'scores-{timestamp}.csv', index=False)
score_df.head()


/tmp/ipykernel_2308/2220774760.py:15: DeprecationWarning: datetime.datetime.utcnow() is deprecated and scheduled for removal in a future version. Use timezone-aware objects to represent datetimes in UTC: datetime.datetime.now(datetime.UTC).
  timestamp = datetime.utcnow().strftime('%Y-%m-%d-%H-00-00')


,word,score
40,trump,47
119,hamas,17
255,gaza,15
465,will,13
20,china,13


## Rank headlines by score

In [4]:
word_scores = dict(score_df[['word','score']].values)
latest['score'] = latest['title'].apply(
    lambda t: sum(
        word_scores.get(w.lower(), 0)
        for w in re.findall(r'[A-Za-z]+', t)
        if len(w) > 1
    )
)
ranked = latest.sort_values('score', ascending=False)
ranked[['score','pubdate','source','title','link']].to_csv('rank.csv', index=False)
ranked[['score','pubdate','source','title','link']].to_csv(archive / f'rank-{timestamp}.csv', index=False)
ranked.head()


,pubdate,source,title,link,score
235,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...,110
167,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...,108
199,2025-10-14-17-30-22 +0000,nypost,"Gaza peace process ‘IS NOT DONE,’ Trump warns ...",https://nypost.com/2025/10/14/us-news/gaza-pea...,106
251,2025-10-14-14-24-22 +0000,nypost,Kamala Harris refuses to name Trump as she lau...,https://nypost.com/2025/10/14/us-news/kamala-h...,105
250,2025-10-14-14-33-51 +0000,nyt,How Trump’s Gaza Peace Plan Presents Hurdles f...,https://www.nytimes.com/2025/10/14/world/middl...,103


## Select top headlines

In [5]:
with open('exclude_sources.txt') as f:
    excluded_sources = {
        line.strip()
        for line in f
        if line.strip() and not line.strip().startswith('#')
    }

top_count = 10
top_rows = []
working = word_scores.copy()
remaining = latest.copy()

while len(top_rows) < top_count and not remaining.empty:
    scored_remaining = remaining.assign(
        score=remaining['title'].apply(
            lambda t: sum(
                working.get(w.lower(), 0)
                for w in re.findall(r'[A-Za-z]+', t)
                if len(w) > 1
            )
        )
    )
    ranked_loop = scored_remaining.sort_values('score', ascending=False)

    if ranked_loop.empty:
        break

    top_story = ranked_loop.iloc[0]
    remaining = remaining.drop(top_story.name)

    if top_story['source'] in excluded_sources:
        continue

    words = set(re.findall(r'[A-Za-z]+', top_story['title'].lower()))
    for word in words:
        working.pop(word, None)

    top_rows.append(top_story[['score', 'pubdate', 'source', 'title', 'link']])

top_df = pd.DataFrame(top_rows)
top_df.to_csv('top.csv', index=False)
top_df.to_csv(archive / f'top-{timestamp}.csv', index=False)
top_df


,score,pubdate,source,title,link
235,110,2025-10-14-15-25-15 +0000,nypost,Hamas won’t give up arms or Gaza control — des...,https://nypost.com/2025/10/14/world-news/hamas...
270,55,2025-10-14-13-00-00 +0000,wsj,"Tariffs, inflation and other threats such as e...",https://www.wsj.com/economy/trade-tensions-wei...
153,49,2025-10-14-19-42-00 +0000,wsj,Maine’s Democratic Gov. Janet Mills officially...,https://www.wsj.com/politics/elections/susan-c...
66,43,2025-10-14-23-18-03 +0000,wapo,U.S. kills 6 alleged drug smugglers in new str...,https://www.washingtonpost.com/national-securi...
160,41,2025-10-14-19-08-39 +0000,bbc,Israel identifies bodies of first four dead ho...,https://www.bbc.com/news/articles/cx27ye11ey2o...
18,40,2025-10-15-02-00-00 +0000,bbc,Government under pressure to release China spy...,https://www.bbc.com/news/articles/cql9v6x4wpzo...
167,39,2025-10-14-18-52-56 +0000,nypost,"Hamas will disarm or ‘we will disarm them,’ Tr...",https://nypost.com/2025/10/14/us-news/hamas-wi...
149,38,2025-10-14-19-55-46 +0000,bbc,Madagascar military says it has seized power a...,https://www.bbc.com/news/articles/cn8xjjdgl8vo...
73,36,2025-10-14-23-01-28 +0000,nypost,Victim bravely tells sicko who raped her in NY...,https://nypost.com/2025/10/14/us-news/you-did-...
232,34,2025-10-14-16-00-00 +0000,wsj,He Works Full Time and Goes to College. Is He ...,https://www.wsj.com/personal-finance/savings/c...


## Create JSON version of top headlines

In [6]:
import pandas as pd
pd.read_csv('top.csv').to_json('top.json', orient='records', indent=2)
